In [56]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data.make_dataset import DICT_VARIABLES
from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

In [58]:
cie_9 = pl.read_excel(
    "../data/external/Esquema_Registro-2023.xlsx",
    sheet_name="Anexo 2",
    read_csv_options={"skip_rows": 5},
)

## 1. Análisis Sociodemográfico de Pacientes Atendidos en el Hospital del Tórax

En este cuadernillo se quiere ver el perfil sociodemográfico de los pacientes hospitalizados
en el Hospital del Tórax. Se utilizará la base de datos pública de egresos hospitalarios DEIS
entre 2001 y 2020. Las preguntas que se quieren responder son:

- ¿Cómo es la distribución geográfica de los pacientes atendidos en el INT?
- ¿Cuál es el rango etario más atendido en el INT?
- ¿Cuál es el género más atendido en el INT?
- ¿Cuál es la previsión más atendida en el INT?
- ¿Cuál es la letra de FONASA más atendida en el INT?

Para responder estas preguntas se realizará una agrupación de los datos con todas las variables
sociodemográficas de interés para el análisis.

In [59]:
# Define todas las variables que se quieren saber

VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS = ["ANO_EGRESO", "REGION_RESIDENCIA", "COMUNA_RESIDENCIA"]
VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS = ["SEXO", "EDAD_CATEGORIA", "PREVISION", "BENEFICIARIO"]
VARIABLE_A_CONTAR = "DIAG1"

agrupacion_total = (
    VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS
    + VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS
    + [VARIABLE_A_CONTAR]
)

Luego, se genera la consulta de la base de datos. Para reducir el coste computacional, se filtra
la base de datos únicamente para los datos del INT. Posteriormente, se agrupa la base de datos
con las variables sociodemográficas de interés y se cuenta la ocurrencia de cada grupo.

In [69]:
desglose_sociodemografico = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .group_by(agrupacion_total)
    .count()
    .sort(["ANO_EGRESO"])
    .collect(streaming=True)
)

desglose_sociodemografico = (
    desglose_sociodemografico.join(
        cie_9, how="left", left_on="COMUNA_RESIDENCIA", right_on="Código Comuna"
    )
    .with_columns(pl.col(["PREVISION", "Nombre Comuna"]).str.to_titlecase())
    .drop("Código Región")
)

In [70]:
display(desglose_sociodemografico.head())

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Nombre Región
i64,i64,i64,str,str,str,str,str,u32,str,str
2001,13,13402,"""Mujer""","""(80, 90]""","""Fonasa""","""B""","""J90X""",1,"""Buin""","""Metropolitana …"
2001,13,13123,"""Hombre""","""(60, 70]""","""Fonasa""","""B""","""A199""",1,"""Providencia""","""Metropolitana …"
2001,13,13113,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""I050""",1,"""La Reina""","""Metropolitana …"
2001,14,14104,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""I350""",1,"""Los Lagos""","""de Los Rios"""
2001,4,4301,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""B679""",1,"""Ovalle""","""de Coquimbo"""


En los resultados se indica el perfil sociodemográfico para cada combinación de variable presente
en la base de datos. A modo de ejemplo, hubo 1 ocurrencia en el 2001 de una paciente de la Región
número 13 y Comuna 13605 que tenía entre 70 a 80 años que tenía PREVISION FONASA Letra B diagnosticada
con el diagnóstico K810.

Uno de los diagnósticos de interés para el Instituto es el de código "C341".

In [62]:
display(desglose_sociodemografico.filter(pl.col("DIAG1") == "C341"))
print(desglose_sociodemografico.filter(pl.col("DIAG1") == "C341").select(pl.col("count").sum()))

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Código Región,Nombre Región
i64,i64,i64,str,str,str,str,str,u32,str,i64,str
2001,13,13128,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1,"""RENCA""",13,"""Metropolitana …"
2001,13,13101,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1,"""SANTIAGO""",13,"""Metropolitana …"
2001,13,13113,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1,"""LA REINA""",13,"""Metropolitana …"
2001,3,3102,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1,"""CALDERA""",3,"""de Atacama"""
2001,8,8101,"""Hombre""","""(60, 70]""","""DESCONOCIDO""",null,"""C341""",1,"""CONCEPCIÓN""",8,"""del Biobío"""
2001,13,13106,"""Mujer""","""(60, 70]""","""FONASA""","""B""","""C341""",1,"""ESTACIÓN CENTR…",13,"""Metropolitana …"
2001,3,3101,"""Hombre""","""(60, 70]""","""FONASA""","""C""","""C341""",1,"""COPIAPO""",3,"""de Atacama"""
2003,13,13110,"""Mujer""","""(50, 60]""","""FONASA""","""B""","""C341""",1,"""LA FLORIDA""",13,"""Metropolitana …"
2003,6,6101,"""Hombre""","""(50, 60]""","""FONASA""","""A""","""C341""",1,"""RANCAGUA""",6,"""del Libertador…"


shape: (1, 1)
┌───────┐
│ count │
│ ---   │
│ u32   │
╞═══════╡
│ 1709  │
└───────┘


Por lo tanto, se muestra cada desglose sociodemográfico de los pacientes atendidos. Además,
se indica que la suma de cada conteo de grupo suma 1709 entre 2001 y 2020. Esto está en concordancia
con los egresos totales (1709) para el diagnóstico mencionado en el periodo mencionado.

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,Nombre Comuna,Código Región,Nombre Región
i64,i64,i64,str,str,str,str,str,u32,str,i64,str
2001,13,13114,"""Hombre""","""(80, 90]""","""Fonasa""","""D""","""I259""",1,"""Las Condes""",13,"""Metropolitana …"
2001,13,13101,"""Mujer""","""(70, 80]""","""Fonasa""","""B""","""I340""",1,"""Santiago""",13,"""Metropolitana …"
2001,13,13101,"""Hombre""","""(70, 80]""","""Isapre""",null,"""K449""",1,"""Santiago""",13,"""Metropolitana …"
2001,12,12101,"""Hombre""","""(70, 80]""","""Fonasa""","""B""","""I351""",1,"""Punta Arenas""",12,"""de Magallanes …"
2001,13,13120,"""Hombre""","""(60, 70]""","""Fonasa""","""B""","""C349""",1,"""Ñuñoa""",13,"""Metropolitana …"
2001,13,13131,"""Hombre""","""(60, 70]""","""Fonasa""","""D""","""J90X""",1,"""San Ramón""",13,"""Metropolitana …"
2001,13,13131,"""Mujer""","""(50, 60]""","""Fonasa""","""A""","""J459""",1,"""San Ramón""",13,"""Metropolitana …"
2001,6,6101,"""Mujer""","""(40, 50]""","""Fonasa""","""A""","""T821""",1,"""Rancagua""",6,"""del Libertador…"
2001,16,16302,"""Hombre""","""(30, 40]""","""Fonasa""","""B""","""J986""",1,"""Coihueco""",16,"""de Ñuble"""
